In [0]:

-- ALTER TABLE apachedataingestion.nr.widetable_5g_du_hr_gold2 ADD COLUMN 	  D_DTM_UTC TIMESTAMP first;


delete from apachedataingestion.nr.widetable_5g_du_hr_gold2 
where CalendarDate >= date_format(date_add(now(), int(:days)),"yyyy-MM-dd") and CalendarDate < date_format(date_add(now(), (int(:days))+1),"yyyy-MM-dd") 
;

insert into apachedataingestion.nr.widetable_5g_du_hr_gold2
with conf_market (select cast(SUB_MARKET_ID AS INT) SUB_MARKET_ID,SUB_MARKET_NAME,REGION,Time_Offset from hive_metastore.default.vz_conf_market)
select 
case 	when B.Time_Offset=-5 then  convert_timezone('America/New_York','UTC',cast(concat(Y.CalendarDate,' ',lpad(Y.hour24,2,0),':00:00') as TIMESTAMP)) 
	 				when B.Time_Offset=-6 then  convert_timezone('America/Chicago','UTC',cast(concat(Y.CalendarDate,' ',lpad(Y.hour24,2,0),':00:00') as TIMESTAMP)) 
	 				when B.Time_Offset=-8 then  convert_timezone('America/Los_Angeles','UTC',cast(concat(Y.CalendarDate,' ',lpad(Y.hour24,2,0),':00:00') as TIMESTAMP)) 
 		end D_DTM_UTC,
Y.* from (
with conf_du (          
			          	select substring(DU_ID,1,10) GNODEB_ID,DU_ID,max(DU_NAME) DU_NAME,max(MARKET_NAME) MARKET_NAME,max(NETWORK_NAME) NETWORK_NAME,max(SUB_MARKET_NAME) SUB_MARKET_NAME,
							cast (replace(substring(DU_ID,1,10),'VZ_','') as bigint) kGNODEB_ID ,cast (replace(DU_ID,'VZ_','') as bigint) kDU_ID			          	
							from hive_metastore.`default`.vz_conf_gnode_du_gcell_silver  where not(length(GNODEB_ID) < 10 or length(du_id) < 14) group by DU_ID
						union all													
						select  distinct concat('VZ_',a.gNB_ID) GNODEB_ID,concat('VZ_',a.DU_ID) DU_ID,						
							'' DU_NAME,'' MARKET_NAME,'' NETWORK_NAME,'' SUB_MARKET_NAME,cast (a.gNB_ID as bigint) kGNODEB_ID ,cast (a.DU_ID as bigint) kDU_ID
							from hive_metastore.`default`.jwk_mapping_live a
							full join (SELECT distinct GNODEB_ID,DU_ID FROM hive_metastore.`default`.vz_conf_gnode_du_gcell_silver) b on concat('VZ_',a.gNB_ID)=b.GNODEB_ID and concat('VZ_',a.DU_ID)=b.DU_ID
							where b.DU_ID is null 
)
/*
with conf_du (          
			          	select substring(DU_ID,1,10) GNODEB_ID,DU_ID,max(DU_NAME) DU_NAME,max(MARKET_NAME) MARKET_NAME,max(NETWORK_NAME) NETWORK_NAME,max(SUB_MARKET_NAME) SUB_MARKET_NAME,
							cast (replace(substring(DU_ID,1,10),'VZ_','') as bigint) kGNODEB_ID ,cast (replace(DU_ID,'VZ_','') as bigint) kDU_ID			          	
							from 
              					(select * from hive_metastore.`default`.vz_conf_gnode_du_gcell_silver union all select * from hive_metastore.`default`.xxx_jwk_vz_conf_gnode_du_gcell_silver_bkp)
              					where not(length(GNODEB_ID) < 10 or length(du_id) < 14) group by DU_ID
						union all													
						select  distinct concat('VZ_',a.gNB_ID) GNODEB_ID,concat('VZ_',a.DU_ID) DU_ID,						
							'' DU_NAME,'' MARKET_NAME,'' NETWORK_NAME,'' SUB_MARKET_NAME,cast (a.gNB_ID as bigint) kGNODEB_ID ,cast (a.DU_ID as bigint) kDU_ID
							from hive_metastore.`default`.jwk_mapping_live a
							full join (SELECT distinct GNODEB_ID,DU_ID FROM hive_metastore.`default`.vz_conf_gnode_du_gcell_silver) b on concat('VZ_',a.gNB_ID)=b.GNODEB_ID and concat('VZ_',a.DU_ID)=b.DU_ID
							where b.DU_ID is null 
)
*/
select 
CalendarDate ,
Hour24 ,
GNODEB_ID,
DU_ID,
MARKET_NAME ,
substring(du_id,4,3) SUB_MARKET_ID,
SUB_MARKET_NAME ,
case when length(du_id)=14 then 
	case when substring(du_id, 7, 1)=0 then 'FR2' 	 when substring(du_id, 7, 1)=9 then 
				case when substring(du_id, 11, 1)=2 then 'FR1/LS3' when substring(du_id, 11, 1)=3 then 'FR1/LS6' 
				end
	end
else '' end NR_TYPE,
max(MaxRRC_users_per_DU) S5NR_MaxRRCconnectedUsersPerDU,
max(EndcAddAtt_DU) EndcAddAtt_DU,
max(AvgRRC_users_per_DU_NUM)   S5NR_AvgRRCconnectedUsersPerDU_Num,
max(AvgRRC_users_per_DU_DENOM) S5NR_AvgRRCconnectedUsersPerDU_Denom,
max(S5NR_F1UPacketLossRate_DL_PCT_Num) S5NR_F1UPacketLossRate_DL_PCT_Num,
max(S5NR_F1UPacketLossRate_DL_PCT_Denom) S5NR_F1UPacketLossRate_DL_PCT_Denom,
max(S5ND_F1UPacketLossRate_DL_PCT_Num) S5ND_F1UPacketLossRate_DL_PCT_Num,
max(S5ND_F1UPacketLossRate_DL_PCT_Denom) S5ND_F1UPacketLossRate_DL_PCT_Denom,
max(S5ND_F1U_PktLoss_DL_SA_ppm_PCT_Num) S5ND_F1U_PktLoss_DL_SA_ppm_PCT_Num,
max(S5ND_F1U_PktLoss_DL_SA_ppm_PCT_Denom) S5ND_F1U_PktLoss_DL_SA_ppm_PCT_Denom,
MAX(S5ND_AvgRRCUsers_SANSA_perDU_Num) S5ND_AvgRRCUsers_SANSA_perDU_Num,
MAX(S5ND_AvgRRCUsers_SANSA_perDU_Denom) S5ND_AvgRRCUsers_SANSA_perDU_Denom,
MAX(S5ND_MaxRRCUsers_SANSA_perDU) S5ND_MaxRRCUsers_SANSA_perDU,
--Added new KPIs on 12/11/2024 JIRA DENG-2416
MAX(F1U_DELAY_NSA_Num) F1U_DELAY_NSA_Num,
MAX(F1U_DELAY_NSA_Denom) F1U_DELAY_NSA_Denom,
MAX(F1_UL_packet_loss_Num) F1_UL_packet_loss_Num,
MAX(F1_UL_packet_loss_Denom) F1_UL_packet_loss_Denom
from ( 
select         
				COALESCE(M0.CalendarDate,T1.CalendarDate,T2.CalendarDate,T3.CalendarDate,T4.CalendarDate,T5.CalendarDate,T6.CalendarDate) CalendarDate,
				COALESCE(M0.Hour24,T1.Hour24,T2.Hour24,T3.Hour24,T4.Hour24,T5.Hour24,T6.Hour24) Hour24,
				COALESCE(M0.GNODEB_ID,T1.GNODEB_ID,T2.GNODEB_ID,T3.GNODEB_ID,T4.GNODEB_ID,T5.GNODEB_ID,T6.GNODEB_ID) GNODEB_ID,
				COALESCE(M0.DU_ID,T1.DU_ID,T2.DU_ID,T3.DU_ID,T4.DU_ID,T5.DU_ID,T6.DU_ID) DU_ID,
				COALESCE(M0.MARKET_NAME,T1.MARKET_NAME,T2.MARKET_NAME,T3.MARKET_NAME,T4.MARKET_NAME,T5.MARKET_NAME,T6.MARKET_NAME) MARKET_NAME,        		        		
				COALESCE(M0.SUB_MARKET_NAME,T1.SUB_MARKET_NAME,T2.SUB_MARKET_NAME,T3.SUB_MARKET_NAME,T4.SUB_MARKET_NAME,T5.SUB_MARKET_NAME,T6.SUB_MARKET_NAME) SUB_MARKET_NAME,     
				DuUeNoMax MaxRRC_users_per_DU,
				EndcAddAtt_DU,
				DuUeNoTot AvgRRC_users_per_DU_NUM,
				DuUeNoCnt AvgRRC_users_per_DU_DENOM,
				(F1UPacketLossCntDL_QCI - F1UPacketOosCntDL_QCI) S5NR_F1UPacketLossRate_DL_PCT_Num ,
				(F1UPacketLossCntDL_QCI + F1UPacketCntDL_QCI - F1UPacketOosCntDL_QCI) S5NR_F1UPacketLossRate_DL_PCT_Denom,
				F1UPacketLossCntDL_QCI						   S5ND_F1UPacketLossRate_DL_PCT_Num,		
				(F1UPacketLossCntDL_QCI + F1UPacketCntDL_QCI)  S5ND_F1UPacketLossRate_DL_PCT_Denom,
				S5ND_F1U_PktLoss_DL_SA_ppm_PCT_Num,
				S5ND_F1U_PktLoss_DL_SA_ppm_PCT_Denom,
				S5ND_AvgRRCUsers_SANSA_perDU_Num,
				S5ND_AvgRRCUsers_SANSA_perDU_Denom,
				S5ND_MaxRRCUsers_SANSA_perDU,
				--Added new KPIs on 12/11/2024 JIRA DENG-2416
				F1uDelayTot F1U_DELAY_NSA_Num,
				F1uDelayCnt F1U_DELAY_NSA_Denom,
				F1URxPacketLossCnt F1_UL_packet_loss_Num,
				(F1URxPacketCnt + F1URxPacketLossCnt) F1_UL_packet_loss_Denom

from (
                  SELECT  Daily CalendarDate,Hourly Hour24,b.du_id,GNODEB_ID,MARKET_NAME, SUB_MARKET_NAME,
	                    sum(DuUeNoTot_count_SUM) DuUeNoTot,
						sum(DuUeNoCnt_count_SUM) DuUeNoCnt,
						Max(DuUeNoMax_count_MAX) DuUeNoMax  
                        -- select *
                      from  apachedataingestion.nr.ueconnectionnumberpergnbdu_nr_hr_silver a
                      inner join conf_DU b on cast(left(REGEXP_REPLACE(a.NE_NAME, '[^0-9]', ''),11) as bigint)=b.kDU_ID 
					  WHERE 5=5 and  Daily  =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd") 
                  GROUP BY Daily,Hourly,b.du_id,GNODEB_ID,MARKET_NAME, SUB_MARKET_NAME
            ) M0         
                  full join  (
					SELECT Daily CalendarDate,Hourly Hour24,b.du_id,GNODEB_ID,MARKET_NAME, SUB_MARKET_NAME,
						sum(EndcAddAtt_DU_count_SUM) EndcAddAtt_DU
                        -- select *
					from apachedataingestion.nr.sgnbadditionperdupergnb_nr_hr_silver  a
					--from usm.nr.sgnbadditionperdupergnb_nr_hr_silver  a
					inner join conf_du b on cast(replace(split(a.LOCATION,'/')[1],'gNB_DU_ID','') as bigint)=b.kDU_ID
					WHERE 5=5 and  Daily  =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd") 
					GROUP BY  Daily,Hourly,b.du_id,GNODEB_ID,MARKET_NAME, SUB_MARKET_NAME
			) T1 on M0.CalendarDate=T1.CalendarDate and M0.Hour24=T1.Hour24 and M0.du_id=T1.du_id 
			full join  (
				SELECT  Daily CalendarDate,Hourly Hour24,b.du_id,GNODEB_ID,MARKET_NAME, SUB_MARKET_NAME,
	                    sum(F1UPacketLossCntDL_QCI_count_SUM) F1UPacketLossCntDL_QCI,
						sum(F1UPacketCntDL_QCI_count_SUM) F1UPacketCntDL_QCI,
						sum(F1UPacketOosCntDL_QCI_count_SUM) F1UPacketOosCntDL_QCI 
                        -- select *
                      from  apachedataingestion.nr.f1udlinterfaceperqci_nr_hr_silver a
                      inner join conf_DU b on cast(left(REGEXP_REPLACE(a.NE_NAME, '[^0-9]', ''),11) as bigint)=b.kDU_ID 
					  WHERE 5=5 and  Daily  =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd") 
					  GROUP BY Daily,Hourly,b.du_id,GNODEB_ID,MARKET_NAME, SUB_MARKET_NAME
			) T2 on M0.CalendarDate=T2.CalendarDate and M0.Hour24=T2.Hour24 and M0.du_id=T2.du_id 
			full join  (
				SELECT  Daily CalendarDate,Hourly Hour24,b.du_id,GNODEB_ID,MARKET_NAME, SUB_MARKET_NAME,
						sum(F1UPacketLossCntDL * power(10, 6))	S5ND_F1U_PktLoss_DL_SA_ppm_PCT_Num,
						sum(F1UPacketCntDL + F1UPacketLossCntDL) S5ND_F1U_PktLoss_DL_SA_ppm_PCT_Denom
                        -- select *
                      from  apachedataingestion.nr.f1udlinterfaceperdu_nr_hr_silver a
                      inner join conf_DU b on cast(left(REGEXP_REPLACE(a.NE_NAME, '[^0-9]', ''),11) as bigint)=b.kDU_ID 
					  WHERE 5=5 and  Daily  =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd") 
					  GROUP BY Daily,Hourly,b.du_id,GNODEB_ID,MARKET_NAME, SUB_MARKET_NAME
			) T3 on M0.CalendarDate=T3.CalendarDate and M0.Hour24=T3.Hour24 and M0.du_id=T3.du_id 
			--Modified silver table for the below KPIs from apachedataingestion.nr.CallNumberinDUNR_nr_hr_silver to apachedataingestion.nr.uenumberperbpu_nr_hr_silver on 2025-02-18 JIRA:DENG-2983
			full join  (
				SELECT  Daily CalendarDate,Hourly Hour24,b.du_id,GNODEB_ID,MARKET_NAME, SUB_MARKET_NAME,
						sum(SpCellUeNumPerBpuTot)	S5ND_AvgRRCUsers_SANSA_perDU_Num,
						sum(SpCellUeNumPerBpuCnt) S5ND_AvgRRCUsers_SANSA_perDU_Denom,
						max(SpCellUeNumPerBpuMax) S5ND_MaxRRCUsers_SANSA_perDU
                        -- select *
                      from  apachedataingestion.nr.uenumberperbpu_nr_hr_silver a
                      inner join conf_DU b on cast(left(REGEXP_REPLACE(a.NE_NAME, '[^0-9]', ''),11) as bigint)=b.kDU_ID 
					  WHERE 5=5 and  Daily  =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd") 
					  GROUP BY Daily,Hourly,b.du_id,GNODEB_ID,MARKET_NAME, SUB_MARKET_NAME
			) T4 on M0.CalendarDate=T4.CalendarDate and M0.Hour24=T4.Hour24 and M0.du_id=T4.du_id 
			full join  (
				SELECT  Daily CalendarDate,Hourly Hour24,b.du_id,GNODEB_ID,MARKET_NAME, SUB_MARKET_NAME,
						sum(F1URxPacketLossCnt) F1URxPacketLossCnt,
						sum(F1URxPacketCnt) F1URxPacketCnt
												
                        -- select *
                      from  apachedataingestion.nr.f1uinterfacepergnbduperqci_nr_hr_silver a
                      inner join conf_DU b on replace(split(a.LOCATION,'/')[1],'gNB_DU_ID','')=b.kDU_ID 
					  WHERE 5=5 and  Daily  =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd") 
					  GROUP BY Daily,Hourly,b.du_id,GNODEB_ID,MARKET_NAME, SUB_MARKET_NAME
			) T5 on M0.CalendarDate=T5.CalendarDate and M0.Hour24=T5.Hour24 and M0.du_id=T5.du_id 
			full join  (
				SELECT  Daily CalendarDate,Hourly Hour24,b.du_id,GNODEB_ID,MARKET_NAME, SUB_MARKET_NAME,
						sum(F1uDelayTot) F1uDelayTot,
						sum(F1uDelayCnt) F1uDelayCnt
                       -- select *
                      from  apachedataingestion.nr.f1udelaymeasurementdataperupcidpergnbduidperqci_nr_hr_silver a
                      inner join conf_DU b on replace(split(a.LOCATION,'/')[1],'gNB_DU_ID','')=b.kDU_ID 
					  WHERE 5=5 and  Daily  =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd") 
					  GROUP BY Daily,Hourly,b.du_id,GNODEB_ID,MARKET_NAME, SUB_MARKET_NAME
			) T6 on M0.CalendarDate=T6.CalendarDate and M0.Hour24=T6.Hour24 and M0.du_id=T6.du_id 
) X            
Group by CalendarDate ,Hour24 ,GNODEB_ID,DU_ID ,MARKET_NAME  ,SUB_MARKET_ID,SUB_MARKET_NAME,NR_TYPE 
) Y
inner join conf_market B on cast(Y.SUB_MARKET_ID AS INT)=B.SUB_MARKET_ID
;




